- Merge All the data which download in kaggle

In [1]:
import pandas as pd
import os
from glob import glob

folder_path = "../2_data/NVDA_2013_2022" 


csv_files = glob(os.path.join(folder_path, "*.csv"))


df_list = []


for file in csv_files:
    try:
        df = pd.read_csv(file)
        df_list.append(df)
    except Exception as e:
        print(f" Error reading {file}: {e}")

merged_df = pd.concat(df_list, ignore_index=True)


merged_df.to_csv("../2_data/merged_nvda.csv", index=False)

- Just run second block of code...

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/q/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/q/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/q/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/q/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/q/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/q/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize

text = "Finally my issue of nltk is resolved"
tokens = word_tokenize(text,language='english', preserve_line=True)
print(tokens)

['Finally', 'my', 'issue', 'of', 'nltk', 'is', 'resolved']


In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_tweet(text):
    text = str(text).lower()
    text = re.sub(r"http[s]?://\S+", "", text)  # drop http (they are just ad)
    text = re.sub(r"www\.\S+", "", text)
    text = re.sub(r"@\w+", "", text)  # drop @
    text = re.sub(r"#", "", text)  # keep hashtag
    text = re.sub(r"[^\w\s]", "", text)  # drop punctuation
    text = re.sub(r"\s+", " ", text).strip()  # drop unnecessary blank
    return text

df = pd.read_csv("../2_data/merged_nvda.csv")
keep_cols = ['body', 'created_at', 'symbols']
existing_cols = [col for col in keep_cols if col in df.columns]
df = df[existing_cols].copy()


df["clean_tweet"] = df["body"].apply(clean_tweet)
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# Keep records containing only NVDA (if symbols are str or list)
df['symbols'] = df['symbols'].astype(str)
df = df[df['symbols'].str.contains("NVDA")]

print(df.head())
df.to_csv("../2_data/nvidia_tweets_cleaned.csv", index=False)

[nltk_data] Downloading package punkt to /Users/q/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/q/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/rn/lc8n68_d7ld6yymqc_lsnrr00000gn/T/ipykernel_31872/436574755.py:24: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../2_data/merged_nvda.csv")


                                                body  \
0         $NVDA going in for some weekly calls here.   
1  NVIDIA&#39;s PT raised by Evercore ISI to $250...   
2  Wells Fargo Maintains Overweight on NVIDIA, Ra...   
3                500 $NVDA AUG2021 $205 Cs trade 3.1   
4                                $NVDA Great volume.   

                 created_at  \
0 2021-08-12 13:46:00+00:00   
1 2021-08-12 13:45:26+00:00   
2 2021-08-12 13:45:12+00:00   
3 2021-08-12 13:43:30+00:00   
4 2021-08-12 13:43:16+00:00   

                                             symbols  \
0  [{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...   
1  [{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...   
2  [{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...   
3  [{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...   
4  [{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...   

                                         clean_tweet  
0           nvda going in for some weekly calls here  
1  nvidia39s pt raised by eve

- Most tweets are just spam, ads...so build a simple filter to get rid of them

In [5]:
import pandas as pd
import re
import nltk
nltk.data.path.append('/Users/q/nltk_data') 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# keyword filter
advertising_keywords = [
    "learn trading", "get alerts", "get ideas", "get updates", "get analysis",
    "welcome to discord", "welcome to the discord group", "bitcoin", "crypto",
    "top analyst price target", "top analyst target price", "top analyst target for next week",
    "🎯", "📈", "📉", "🚀"
]

def is_advertising(text: str) -> bool:
    """if contains https or any keywords in filter，take it as ad"""
    if not isinstance(text, str):
        return False
    txt = text.lower()
    if 'https' in txt:
        return True
    return any(keyword in txt for keyword in advertising_keywords)

def clean_tweet(text: str) -> str:
    text = str(text).lower()
    text = re.sub(r"http[s]?://\S+", "", text)  # drop links
    text = re.sub(r"www\.\S+", "", text)        # drop links
    text = re.sub(r"@\w+", "", text)            # drop @mention
    text = re.sub(r"#", "", text)               # drop # but keep hashtag text
    text = re.sub(r"[^\w\s]", "", text)         # drop punctuation and Emoji
    text = re.sub(r"\s+", " ", text).strip()    # drop unnecessary blank

    tokens = word_tokenize(text)
    cleaned = [stemmer.stem(w) for w in tokens if w not in stop_words]
    return " ".join(cleaned)

df = pd.read_csv("../2_data/merged_nvda.csv")

# filter
before_count = len(df)
df = df[~df["body"].apply(is_advertising)].copy()
after_count = len(df)
print(f"total：{before_count}，after filtering：{after_count}，drop：{before_count - after_count}")

# datetime
df["clean_tweet"] = df["body"].apply(clean_tweet)
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce").dt.date

df.to_csv("../2_data/nvidia_tweets_filtered_cleaned.csv", index=False)
print("save：nvidia_tweets_filtered_cleaned.csv")

[nltk_data] Downloading package punkt to /Users/q/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/q/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/rn/lc8n68_d7ld6yymqc_lsnrr00000gn/T/ipykernel_31872/2154528657.py:46: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../2_data/merged_nvda.csv")


total：584181，after filtering：513285，drop：70896
save：nvidia_tweets_filtered_cleaned.csv
